In [1]:
1+1

2

In [3]:
# 패키지 설치
# pip install langchain-core langchain-openai langchain-teddynote

from langchain_openai import ChatOpenAI
from langchain.memory import ConversationTokenBufferMemory

# 최신 LLM 사용 (gpt-4.1-mini)
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# 토큰 기준으로 대화 맥락 관리 (최대 150 토큰)
memory = ConversationTokenBufferMemory(
    llm=llm,
    max_token_limit=150,
    return_messages=True,
)

# 대화 저장 예시
memory.save_context(
    inputs={"human": "안녕하세요, 설치 방법을 알려주세요."},
    outputs={"ai": "안녕하세요! 기계 모델명을 알려주시면 안내드리겠습니다."},
)

# 현재 저장된 대화 확인
print(memory.load_memory_variables({})["history"])

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_8708\248015937.py:11: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationTokenBufferMemory(


[HumanMessage(content='안녕하세요, 설치 방법을 알려주세요.', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요! 기계 모델명을 알려주시면 안내드리겠습니다.', additional_kwargs={}, response_metadata={})]


In [4]:
# 패키지 설치
# pip install langchain-core langchain-openai langgraph

from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage
from typing import Dict, Any

# 최신 LLM 사용 (gpt-4o-mini 권장)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


# 메시지 관리를 위한 노드 함수
def chatbot_node(state: MessagesState) -> Dict[str, Any]:
    """채팅봇 응답을 생성하는 노드"""
    messages = state["messages"]

    # 토큰 제한 관리 (최대 150 토큰 상당)
    # 간단한 메시지 수 제한으로 구현 (실제로는 토큰 계산 필요)
    max_messages = 6  # 약 150 토큰에 해당하는 메시지 수

    if len(messages) > max_messages:
        # 최근 메시지만 유지
        messages = messages[-max_messages:]

    # LLM으로 응답 생성
    response = llm.invoke(messages)

    return {"messages": [response]}


# LangGraph 설정
def create_chatbot_graph():
    """메모리를 가진 채팅봇 그래프 생성"""

    # 상태 그래프 초기화
    workflow = StateGraph(MessagesState)

    # 채팅봇 노드 추가
    workflow.add_node("chatbot", chatbot_node)

    # 시작점과 종료점 설정
    workflow.set_entry_point("chatbot")
    workflow.set_finish_point("chatbot")

    # 메모리 저장소 설정
    memory = MemorySaver()

    # 그래프 컴파일 (메모리 포함)
    app = workflow.compile(checkpointer=memory)

    return app


# 채팅봇 인스턴스 생성
chatbot_app = create_chatbot_graph()


# 대화 저장 및 실행 예시
def save_and_run_conversation():
    """대화를 저장하고 실행하는 함수"""

    # 고유한 스레드 ID (사용자 세션별로 구분)
    thread_config = {"configurable": {"thread_id": "user_session_1"}}

    # 첫 번째 대화
    user_message = "안녕하세요, 설치 방법을 알려주세요."

    result1 = chatbot_app.invoke(
        {"messages": [HumanMessage(content=user_message)]}, config=thread_config
    )

    print("첫 번째 응답:")
    print(result1["messages"][-1].content)
    print("\n" + "=" * 50 + "\n")

    # 두 번째 대화 (이전 대화 기억)
    user_message2 = "어떤 기계 모델에 대해 물어봤나요?"

    result2 = chatbot_app.invoke(
        {"messages": [HumanMessage(content=user_message2)]}, config=thread_config
    )

    print("두 번째 응답 (이전 대화 기억):")
    print(result2["messages"][-1].content)
    print("\n" + "=" * 50 + "\n")

    # 현재 저장된 모든 메시지 확인
    print("현재 대화 히스토리:")
    for i, message in enumerate(result2["messages"]):
        role = "Human" if isinstance(message, HumanMessage) else "AI"
        print(f"{i+1}. {role}: {message.content}")


# 실행
save_and_run_conversation()

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.graph import MessagesState, StateGraph, START, END

# LLM 준비
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=1)


# 상태 정의 (메시지와 요약)
class State(MessagesState):
    messages: list
    summary: str


# 대화 길이 체크 함수
def should_continue(state: State):
    if len(state["messages"]) > 6:
        return "summarize_conversation"
    return END


# 요약 노드
def summarize_conversation(state: State):
    summary = state.get("summary", "")
    summary_message = (
        f"This is the conversation summary so far:\n{summary}\n\nPlease extend the summary by incorporating any new messages."
        if summary
        else "Please create a concise summary of the conversation so far."
    )
    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = llm.invoke(messages)
    # 최근 2개 메시지만 남기고 나머지는 삭제
    return {"summary": response.content, "messages": state["messages"][-2:]}


# 워크플로우 그래프 구성
workflow = StateGraph(State)
workflow.add_node(
    "conversation", lambda state: {"messages": [llm.invoke(state["messages"])]}
)
workflow.add_node("summarize_conversation", summarize_conversation)
workflow.add_edge(START, "conversation")
workflow.add_conditional_edges("conversation", should_continue)
workflow.add_edge("summarize_conversation", END)
app = workflow.compile()

# 실제 대화 진행 시, 메시지가 많아지면 자동으로 요약이 생성되고, 요약+최근 메시지만 남음